In [1]:
import pandas as pd


In [2]:
#Loading . data
metadata = pd.read_csv('movies_metadata.csv')

/Users/daywalker/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
#calculating the c
c= metadata['vote_average'].mean()
print(c)
#average rating of a movie

5.618207215134185


In [5]:
#calculate the minimum number of votes to be required in the chart , that is m
m =metadata['vote_count'].quantile(.90)
m

160.0

In [6]:
# filtering the qualified movies into new dataframe
new_metadata= metadata.copy().loc[metadata['vote_count']>=m]
new_metadata.shape

(4555, 24)

In [7]:
#function for weighted rating

def weighted_rating(x,m=m,c=c):
    v=x['vote_count']
    R=x['vote_average']
    # Imdb formula for calculation
    return (v/(v+m) *R) + (m/(m+v)* c)

In [8]:
new_metadata['score']= new_metadata.apply(weighted_rating, axis=1)

In [9]:
# sorting the movies 

new_metadata =new_metadata.sort_values('score',ascending=False)

In [10]:
new_metadata[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


# Content Based recommendor system


plot based

In [13]:
# plot the overview of the movie
metadata['overview'].head(3)

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
Name: overview, dtype: object

In [14]:
# Import the tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
#defining the tf-idf vectorizer. Remove the stop words such as "the", "a"
tfidf = TfidfVectorizer(stop_words="english")

In [16]:
#Replacing the Nans with empty string
metadata['overview']= metadata['overview'].fillna('')

In [17]:
#constructing the tf-idf  matrix by fitting and transforming the data
tfidf_matrix= tfidf.fit_transform(metadata['overview'])
print(tfidf_matrix.shape)
tfidf_matrix


(45466, 75827)


<45466x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

In [18]:
# import linear_kernal
from sklearn.metrics.pairwise import linear_kernel

#computing the cosine similarity
cosine_simi =linear_kernel(tfidf_matrix,tfidf_matrix)

In [19]:
#constructing the reverse map of indices and movie titles
indices = pd.Series(metadata.index,index=metadata['title']).drop_duplicates()
indices

title
Toy Story                               0
Jumanji                                 1
Grumpier Old Men                        2
Waiting to Exhale                       3
Father of the Bride Part II             4
Heat                                    5
Sabrina                                 6
Tom and Huck                            7
Sudden Death                            8
GoldenEye                               9
The American President                 10
Dracula: Dead and Loving It            11
Balto                                  12
Nixon                                  13
Cutthroat Island                       14
Casino                                 15
Sense and Sensibility                  16
Four Rooms                             17
Ace Ventura: When Nature Calls         18
Money Train                            19
Get Shorty                             20
Copycat                                21
Assassins                              22
Powder                      

In [20]:
# function that takes  in movie title and outputs the most similar movies
def plot_recommendations(title,cosine_simi=cosine_simi):
    
    #get the index of the movie title
    idx = indices[title]
    
    #get the pair wise similarity scores of all movies with that movie
    
    sim_scores =list(enumerate(cosine_simi[idx]))
    
    #sort the movies based on the similarity scores
    sim_scores =sorted(sim_scores, key =lambda x:x[1],reverse =True)
    
    sim_scores =sim_scores[1:11]
    
    #get the movie indices
    movies_idx =[i[0] for i in sim_scores]
    
    #return the movies similar
    
    return metadata['title'].iloc[movies_idx]

In [21]:
plot_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [22]:
#credits geners and keywords based recommendation system


In [23]:
#Loading the keywords and  credits
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')


In [24]:
# cleaning the data because at the rows incorrect data is present
print(metadata.iloc[19730])
metadata = metadata.drop([19730, 29503, 35587]) # dropped the rows

adult                                                   - Written by Ørnås
belongs_to_collection                                             0.065736
budget                                    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
genres                   [{'name': 'Carousel Productions', 'id': 11176}...
homepage                 [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
id                                                              1997-08-20
imdb_id                                                                  0
original_language                                                    104.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                                             NaN
poster_path                                                   Midnight Man
production_companies                                                 False
production_countries     

In [25]:
# for merging the data convert the id's into integer data types
keywords['id'] =keywords['id'].astype("int")
credits['id'] =credits['id'].astype("int")
metadata['id'] =metadata['id'].astype("int")

In [26]:
#merging the keywords  and credits with metadata
metadata = metadata.merge(credits, on ='id')
metadata = metadata.merge(keywords, on ='id')

In [27]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [28]:
#Parse the stringfied features into their corresponding python objects
from ast import literal_eval

features =['cast','crew','keywords','genres']

for feature in features:
    metadata[feature]= metadata[feature].apply(literal_eval)

In [29]:
import numpy as np

#get the directors name from the crew feature
def get_director(x):
        for i in x:
            if i['job']=='Director':
                return i["name"]
        return np.nan
        

In [30]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [31]:
metadata['director'] = metadata['crew'].apply(get_director) 

features = ['cast','keywords','genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [32]:
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [33]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [34]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [35]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' '+ ' '.join(x['cast'])+ ' '+x['director']+ ' '+' '.join(x['genres'])

In [36]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [38]:
metadata['soup'].head(1)

0    jealousy toy boy tomhanks timallen donrickles ...
Name: soup, dtype: object

In [39]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [40]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [41]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])


In [43]:
plot_recommendations('The Dark Knight Rises', cosine_sim2)

12589      The Dark Knight
10210        Batman Begins
9311                Shiner
9874       Amongst Friends
7772              Mitchell
516      Romeo Is Bleeding
11463         The Prestige
24090            Quicksand
25038             Deadfall
41063                 Sara
Name: title, dtype: object